## Develop a factory manager (agent) for the SCM world

This tutorial describes how to develop an agent for the SCM world, test it, and submit it to the ANAC 2019 SCM league.

The first step is to install negmas

```bash
pip install negmas
```
Once you have this library installed, you can start developing your factory manager:

In [30]:
from negmas.apps.scml import FactoryManager

class MyFactoryManager(FactoryManager):
    """My factory manager"""

Try to make an object of your newly created factory manager. We will fail, do not panic:

In [31]:
try:
    incorrect = FactoryManager()
except TypeError as e:
    print(e)

Can't instantiate abstract class FactoryManager with abstract methods on_negotiation_request


You are told that you cannot instantiate your newly created class as did not implement the abstract (required) method ```on_negotiation_request```. That is actually the ONLY abstract method you need to implement.

In [32]:
class MyFactoryManager(FactoryManager):
    """My factory manager"""
    
    def on_negotiation_request(self, cfp: "CFP", partner: str):
        return None

As the [documentation](http://www.yasserm.com/negmas/negmas.apps.scml.html?highlight=on_negotiation_request#negmas.apps.scml.SCMLAgent.on_negotiation_request) states, this function is called whenever your factory manager receives a request from another agent to negotiate. You can either return `None` if you do not want to accept this negotiation or create  a `Negotiator` that represents your agent in it.


Your do-nothing agent is almost ready. Let's try it:

In [33]:
from negmas.apps.scml.utils import anac2019_world
from negmas.apps.scml import GreedyFactoryManager

world = anac2019_world(competitors=(MyFactoryManager, GreedyFactoryManager), n_steps=10, n_greedy_per_level=5
                      , n_factories_per_level=11, n_intermediate=3
                      , default_signing_delay=0, max_storage=1000000
                      , max_insurance_premium=100, n_retrials=5)

You have now a world with the settings for ANAC 2019 competition. Let's run it

In [34]:
world.run()

What just happened?

In [35]:
world.winners

[greedy_4_19]

The property `stats` in `World` gives you several statistics about the world for every time-step of the simulation.

Let's try to run a tournament with this do-nothing agent against the built-in greedy agent:

In [36]:
from negmas.apps.scml.utils import anac2019_tournament

results = anac2019_tournament(competitors=(MyFactoryManager, GreedyFactoryManager)
                              , n_runs=4, n_steps=10, parallelism='serial')

TypeError: 'tuple' object does not support item assignment

You can see the scores that each individual factory manager got:

In [ ]:
results.scores

You can also check the total scores for each factory manager type:

In [ ]:
results.total_scores

If you want, you can check if these differences are statistically significant using a t-test:

In [ ]:
results.ttest

So the greedy factory manager is better than the do-nothing factory manager getting a total score of 2878 compared with only 367 for the do-nothing factory manager. Moreover, this difference is statistically significant as the p-value is 0.041 < 0.05.

Or, let's just find out who was the winner:

In [ ]:
results.winner

and what was its to score:

In [ ]:
results.winner_score

Now, you completed the development of your factory manager, tested it by running it in worlds and tournaments, what exactly should you do to participate in the SCM league @ ANAC 2019:

You need to submit the following items:

- Names of all members of the team with their affiliations and email addresses
- Either a single python file with the whole implementation of your agent with any supporting code or a zip file with a single folder containing your code. In the later case, you will need to indicate the class name of your factory manager. Any factory manager names are accepted except (Insurance, Bank, MFactoryManager, CFactoryManager).
- A 2-pages academic report about your factory manager. Please check the submission website for details about this report.

That is it folks!
You can now start developing your own factory manager. Have fun.